# Acoustic Maze Navigation - Wave Propagation Test

**Goal**: Verify k-Wave simulation works correctly by visualizing realistic sound propagation in a maze.

## What we're testing:
1. Sound source emits waves
2. Waves propagate through air
3. Walls reflect/absorb sound
4. Sound reaches ALL parts of the maze (no dead zones)
5. Wave patterns are visible and realistic

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from kwave.kgrid import kWaveGrid
from kwave.kmedium import kWaveMedium
from kwave.ksource import kSource
from kwave.ksensor import kSensor
from kwave.kspaceFirstOrder2D import kspaceFirstOrder2D
from kwave.options.simulation_options import SimulationOptions
from kwave.options.simulation_execution_options import SimulationExecutionOptions
from kwave.utils.signals import tone_burst

## 1. Configuration

In [ ]:
# Grid settings
Nx, Ny = 64, 64  # Grid size (keep small for speed)
dx = dy = 1e-2   # 1 cm resolution

# Physical constants
c_air = 343.0      # Sound speed in air (m/s)
rho_air = 1.2      # Air density (kg/m³)
c_wall = 220.0     # Sound speed in walls (slower = more reflection)
alpha_power = 1.5  # Frequency power law for absorption

# Source settings
f0 = 2000.0        # 2 kHz frequency
num_cycles = 6     # Tone burst cycles
sim_duration = 4e-3  # 4 ms simulation

# Wall absorption (dB per cm at f0)
wall_absorption_db_per_cm = 1.0

print(f"Grid: {Nx}×{Ny} = {Nx*Ny} points")
print(f"Resolution: {dx*100:.1f} cm")
print(f"Physical size: {Nx*dx:.2f}m × {Ny*dy:.2f}m")
print(f"Source: {f0/1000:.1f} kHz, {num_cycles} cycles")
print(f"Simulation: {sim_duration*1000:.1f} ms")

## 2. Create Simple Maze

Using random rectangular obstacles (like gen_envi.py)

In [ ]:
# Create maze with random rectangular blocks
maze = np.zeros((Nx, Ny), dtype=bool)

# Add random blocks
np.random.seed(42)
block_size = 8
num_blocks = 12

for _ in range(num_blocks):
    # Random position on grid
    y0 = np.random.randint(0, Nx // block_size) * block_size
    x0 = np.random.randint(0, Ny // block_size) * block_size
    
    # Add block
    h, w = block_size, block_size
    if y0 + h <= Nx and x0 + w <= Ny:
        maze[y0:y0+h, x0:x0+w] = True

print(f"Maze created: {100*maze.mean():.1f}% walls")

# Visualize maze
plt.figure(figsize=(6, 6))
plt.imshow(maze.T, origin='lower', cmap='binary')
plt.title('Maze Layout (Black = Walls)')
plt.xlabel('X')
plt.ylabel('Y')
plt.axis('image')
plt.tight_layout()
plt.show()

## 3. Setup k-Wave Simulation

Create medium with acoustic properties

In [ ]:
# Create grid
kgrid = kWaveGrid([Nx, Ny], [dx, dy])
kgrid.makeTime(c=c_air, cfl=0.3, t_end=sim_duration)

print(f"Time steps: {kgrid.Nt}")
print(f"Time step: {kgrid.dt*1e6:.2f} µs")

# Create medium properties
sound_speed = np.full((Nx, Ny), c_air, dtype=np.float32)
sound_speed[maze] = c_wall

density = np.full((Nx, Ny), rho_air, dtype=np.float32)

# Absorption (walls absorb sound)
alpha_coeff = np.zeros((Nx, Ny), dtype=np.float32)
alpha_coeff[maze] = wall_absorption_db_per_cm / ((f0 / 1e6) ** alpha_power)

medium = kWaveMedium(
    sound_speed=sound_speed,
    density=density,
    alpha_coeff=alpha_coeff,
    alpha_power=alpha_power
)

# Visualize sound speed
plt.figure(figsize=(7, 6))
plt.imshow(sound_speed.T, origin='lower', cmap='viridis')
plt.title('Sound Speed Map')
plt.xlabel('X (grid points)')
plt.ylabel('Y (grid points)')
plt.colorbar(label='Sound Speed (m/s)')
plt.axis('image')
plt.tight_layout()
plt.show()

## 4. Create Source and Sensor

- **Source**: Point source emitting tone burst
- **Sensor**: Record pressure at ALL grid points to visualize wave propagation

In [ ]:
# Source position (corner of maze)
source_y = int(Nx * 0.9)
source_x = int(Ny * 0.9)

# Make sure source is not in wall
if maze[source_y, source_x]:
    print("⚠ Source in wall! Finding air position...")
    air_positions = np.argwhere(~maze)
    source_y, source_x = air_positions[-1]  # Take last air position

print(f"Source position: ({source_y}, {source_x})")
print(f"Source in wall: {maze[source_y, source_x]}")

# Create source
source = kSource()
source_mask = np.zeros((Nx, Ny), dtype=bool)
source_mask[source_y, source_x] = True
source.p_mask = source_mask

# Generate tone burst signal
source.p = tone_burst(1 / kgrid.dt, f0, num_cycles)

print(f"Source signal: {len(source.p)} samples")
print(f"Signal duration: {len(source.p) * kgrid.dt * 1000:.2f} ms")

# Create sensor - record FULL pressure field
sensor = kSensor(
    mask=np.ones((Nx, Ny), dtype=bool),  # Record everywhere!
    record=['p_final', 'p_rms']  # Final pressure and RMS
)

print(f"\nSensor recording {sensor.mask.sum()} points")

## 5. Run k-Wave Simulation

This will take ~30-60 seconds

In [ ]:
# Simulation options
sim_options = SimulationOptions(
    save_to_disk=True,
    pml_inside=False,
    pml_size=10
)

exec_options = SimulationExecutionOptions(
    is_gpu_simulation=False,
    show_sim_log=True  # Show progress
)

print("Running k-Wave simulation...")
print("This will take ~30-60 seconds\n")

sensor_data = kspaceFirstOrder2D(
    kgrid=kgrid,
    source=source,
    sensor=sensor,
    medium=medium,
    simulation_options=sim_options,
    execution_options=exec_options
)

print("\n✓ Simulation complete!")
print(f"Data keys: {list(sensor_data.keys())}")

## 6. Visualize Wave Propagation

Show how sound propagates through the maze

In [ ]:
# Reshape data to 2D grid
p_final = np.reshape(sensor_data['p_final'], (Nx, Ny), order='F')
p_rms = np.reshape(sensor_data['p_rms'], (Nx, Ny), order='F')

print(f"Final pressure range: [{p_final.min():.3f}, {p_final.max():.3f}]")
print(f"RMS pressure range: [{p_rms.min():.6f}, {p_rms.max():.6f}]")

# Check if sound reached borders
border_rms = np.concatenate([
    p_rms[0, :],   # Top
    p_rms[-1, :],  # Bottom
    p_rms[:, 0],   # Left
    p_rms[:, -1]   # Right
])

print(f"\nBorder RMS mean: {border_rms.mean():.6f}")
print(f"Border RMS max: {border_rms.max():.6f}")

if border_rms.max() > p_rms.max() * 0.01:
    print("✓ Sound reached borders!")
else:
    print("⚠ Sound may not have reached borders (increase sim_duration)")

In [ ]:
# Plot Final Pressure Field (snapshot at end of simulation)
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Final pressure
im1 = axes[0].imshow(p_final.T, origin='lower', cmap='RdBu_r', 
                     vmin=-p_final.max()*0.8, vmax=p_final.max()*0.8)
axes[0].contour(maze.T, levels=[0.5], colors='black', linewidths=1.5)
axes[0].scatter([source_x], [source_y], s=200, c='red', marker='*', 
               edgecolors='black', linewidths=1, label='Source', zorder=10)
axes[0].set_title('Final Pressure Field (Wave Pattern)', fontsize=14)
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')
axes[0].legend(loc='upper left')
axes[0].axis('image')
plt.colorbar(im1, ax=axes[0], label='Pressure')

# RMS pressure (shows energy distribution)
im2 = axes[1].imshow(p_rms.T, origin='lower', cmap='hot')
axes[1].contour(maze.T, levels=[0.5], colors='cyan', linewidths=1.5)
axes[1].scatter([source_x], [source_y], s=200, c='red', marker='*',
               edgecolors='black', linewidths=1, label='Source', zorder=10)
axes[1].set_title('RMS Pressure (Energy Distribution)', fontsize=14)
axes[1].set_xlabel('X')
axes[1].set_ylabel('Y')
axes[1].legend(loc='upper left')
axes[1].axis('image')
plt.colorbar(im2, ax=axes[1], label='RMS Pressure')

plt.tight_layout()
plt.show()

## 7. Analysis

Check for acoustic dead zones

In [ ]:
# Find air regions with very low RMS (dead zones)
air_mask = ~maze
p_rms_air_only = p_rms.copy()
p_rms_air_only[maze] = np.nan  # Ignore walls

threshold = p_rms.max() * 0.01  # 1% of max
dead_zones = (p_rms < threshold) & air_mask

num_air_cells = air_mask.sum()
num_dead_zones = dead_zones.sum()

print(f"Total air cells: {num_air_cells}")
print(f"Dead zone cells: {num_dead_zones} ({100*num_dead_zones/num_air_cells:.1f}%)")

if num_dead_zones / num_air_cells > 0.1:
    print("\n⚠ WARNING: >10% of air is dead zones!")
    print("Solutions:")
    print("  - Increase sim_duration (let waves propagate longer)")
    print("  - Move source to center")
    print("  - Reduce wall density")
else:
    print("\n✓ Good coverage! Sound reaches most of the maze.")

# Visualize dead zones
plt.figure(figsize=(8, 7))
plt.imshow(p_rms_air_only.T, origin='lower', cmap='hot')
plt.contour(dead_zones.T, levels=[0.5], colors='lime', linewidths=2, 
           linestyles='dashed')
plt.contour(maze.T, levels=[0.5], colors='white', linewidths=1)
plt.scatter([source_x], [source_y], s=200, c='red', marker='*',
           edgecolors='black', linewidths=1)
plt.title(f'Dead Zones (Green) - {100*num_dead_zones/num_air_cells:.1f}% of air', 
         fontsize=14)
plt.xlabel('X')
plt.ylabel('Y')
plt.colorbar(label='RMS Pressure')
plt.axis('image')
plt.tight_layout()
plt.show()

## ✓ Next Steps

If the above shows:
1. ✓ Clear wave patterns in final pressure
2. ✓ RMS shows energy distribution
3. ✓ Sound reaches borders
4. ✓ Minimal dead zones (<10%)

Then the acoustic simulation is working correctly! 

**Next**: Add microphone array and compute spectrograms for navigation.